## Imports

In [1]:
import sys, os, glob, time , corner
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0,'/home/isha/gw_detection_ias')
import utils as pipeline_utils
import time

from importlib import reload

# cogwheel imports
sys.path.append('/home/isha/PopulationSelectionEffects/cogwheel_population_inference/cogwheel')
import cogwheel
from cogwheel import utils , data, sampling, gw_plotting, posterior, prior

from cogwheel.factorized_qas import IntrinsicParametersPrior, MarginalizedRelativeBinningLikelihood
from cogwheel.population_inference.test_combined_priors import TestPrior
from cogwheel.population_inference.test_combined_priors import FixedTestPopulationPrior, FixedTestPrior
from cogwheel.population_inference.test_combined_priors import FixedDistancePrior, FixedInclinationPrior, GaussianTestPrior

from cogwheel.population_inference.injection_priors import InjectionPrior, InjectionMassPrior
from cogwheel.population_inference.population_models import ParametrizedPrior, CombinedParametrizedPrior, HyperPrior
from cogwheel.population_inference.population_likelihood import PopulationLikelihood
from cogwheel.gw_prior import extrinsic

from cogwheel.prior import Prior, UniformPriorMixin, IdentityTransformMixin, CombinedPrior

/home/isha/miniconda3/envs/cogwheel_pop_latest/lib/python3.9/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS
/tmp/ipykernel_1694326/2061227732.py:17: DeprecationWarning: 
The `factorized_qas` subpackage is deprecated.
The preferred way of marginalizing extrinsic parameters is now
`cogwheel.likelihood.MarginalizedExtrinsicLikelihoodQAS`
coupled to e.g. `gw_prior.IntrinsicAlignedSpinIASPrior`.
See  `tutorials/

In [9]:
# import os
# print(os.environ['CONDA_DEFAULT_ENV'])

## Load injection summary data

In [2]:
injs_summary = pd.read_pickle('injections_summary.pkl')

In [3]:
injs_summary.keys()

Index(['t_rec_h', 't_rec_l', 'rhosq_rec_h', 'rhosq_rec_l', 'normfac_h',
       'normfac_l', 'dphi_rec', 'sb_rec', 'inds', 'cosiota', 'costhetanet',
       'phinet_hat', 't_refdet', 'psi', 'phi_ref_hat', 'm1_source', 'cum_q',
       'd_hat', 'chieff', 'cumchidiff', 'iota', 'ra', 'dec', 't_geocenter',
       'phi_ref', 'm1', 'm2', 'd_luminosity', 's1z', 's2z', 's1x', 's1y',
       's2x', 's2y', 'l1', 'l2', 'f_ref', 'm2_source', 'q', 'mchirp', 'mtot',
       'lnq', 'q1', 'eta', 'chia', 's1', 's1costheta', 's1theta', 's1phi',
       's2', 's2costheta', 's2theta', 's2phi', 'chip', 'phi_H', 'tgps',
       'mchirp_id', 'dt_linfree', 'ln_pdet_fid', 'campaign', 'pdet_fid',
       'dt_closest_inj', 'ifar', 'ifar_no_coherent_score'],
      dtype='object')

### testing dataframe -> dictionary 

In [4]:
dict_example = {'a':np.asarray([1,2,3,4,5]), 'b':np.asarray([6,7,8,9,10]), 'c':np.asarray([11,12,13,14, 15])}
df_example = pd.DataFrame(dict_example)
df_example[np.array(['a','b'])]

dict_of_arrays = {col: df_example[col].to_numpy() for col in df_example.columns}
display(df_example)
print(dict_of_arrays)

,a,b,c
0,1,6,11
1,2,7,12
2,3,8,13
3,4,9,14
4,5,10,15


{'a': array([1, 2, 3, 4, 5]), 'b': array([ 6,  7,  8,  9, 10]), 'c': array([11, 12, 13, 14, 15])}


## Creating Prior Class Instances

In [5]:
parentdir = '/home/isha/PopulationSelectionEffects/output/'

# grab the psds from any exisiting event - we will chnage the event name later
eventname = 'GW190521'

combined_pop_prior = FixedTestPopulationPrior(f_ref=20) # combined prior
pop_prior = GaussianTestPopulationPrior() # population prior
#test_prior = TestPrior(detector_pair='HL', tgps=0.0, ref_det_name='H1', f_avg='10',f_ref=20)
#fixed_test_prior = FixedTestPrior(f_ref=20)
#inj_prior=InjectionPrior(detector_pair='HL', tgps=0.0, ref_det_name='H1', f_avg='10', m1_source_range=(3,10))
#,f_ref=20)
#inj_mass_prior = InjectionMassPrior(tgps=0.0, ref_det_name='H1',m1_source_range=(3,10))
inj_prior=InjectionPrior(detector_pair='HL', tgps=0.0, ref_det_name='H', f_avg=10, m1_source_range=(3,10),f_ref=20)

In [6]:
pop_prior.hyperparam_range_dic

{'lambda1': (10, 10), 'lambda2': (-10, 10)}

In [7]:
# display the pop objects
combined_pop_prior, pop_prior

(FixedTestPopulationPrior(m1, m2) → [iota, ra, dec, t_geocenter, psi, phi_ref, m1, m2, s1z, s2z, s1x_n, s1y_n, s2x_n, s2y_n, l1, l2, f_ref, d_luminosity],
 GaussianTestPopulationPrior(m1, m2) → [m1, m2])

In [8]:
inj_prior.subpriors

[IsotropicInclinationPrior(cosiota) → [iota],
 IsotropicSkyLocationPrior(costhetanet, phinet_hat | iota) → [ra, dec],
 UniformTimePrior(t_refdet | ra, dec) → [t_geocenter],
 UniformPolarizationPrior(psi) → [psi],
 UniformPhasePrior(phi_ref_hat | iota, ra, dec, psi, t_geocenter) → [phi_ref],
 InjectionMassPrior(m1_source, cum_q, d_hat | ra, dec, psi, iota) → [m1, m2, d_luminosity],
 UniformEffectiveSpinPrior(chieff, cumchidiff | m1, m2) → [s1z, s2z],
 ZeroInplaneSpinsPrior() → [s1x_n, s1y_n, s2x_n, s2y_n],
 ZeroTidalDeformabilityPrior() → [l1, l2],
 FixedReferenceFrequencyPrior() → [f_ref]]

## Load PE samples

In [4]:
#read pe samples from saved 
samples_root_dir = '/home/isha/PopulationSelectionEffects/cogwheel_population_inference/cogwheel/cogwheel/population_inference/PE_samples10'
list_events = ['GW150914', 'GW151012', 'GW151216', 
               'GW151226', 'GW170104', 'GW170121', 'GW170202', 'GW170304', 'GW170403', 'GW170425']
all_events_pe = []
for event in list_events:
    event_file_name = os.path.join(samples_root_dir, event+'.feather')
    samples = pd.read_feather(event_file_name)
    all_events_pe.append(samples)

In [5]:
all_events_pe[0].keys()
#all_events_pe[0]['lnl']

Index(['mchirp', 'lnq', 'chieff', 'cumchidiff', 'costheta_jn', 'phi_jl_hat',
       'phi12', 'cums1r_s1z', 'cums2r_s2z', 'psi', 'costhetanet', 'phinet_hat',
       't_refdet', 'phi_ref_hat', 'weights', 'f_ref', 'm1', 'm2', 's1z', 's2z',
       'iota', 's1x_n', 's1y_n', 's2x_n', 's2y_n', 'ra', 'dec', 't_geocenter',
       'phi_ref', 'l1', 'l2', 'd_luminosity', 'lnl', 'lnl_aux_H', 'lnl_aux_L',
       'cosiota', 'm1_source', 'cum_q', 'd_hat'],
      dtype='object')

## Compute Likelihood

In [17]:
# now compute likelihood
injections_summary_dict = {'Ninj':20000, 'recovered_injections':injs_summary, 'pastro_func':np.ones(10)}
R0 = 70

likelihood_object = PopulationLikelihood(combined_pop_prior, injections_summary_dict, all_events_pe, R0, injection_population_model=inj_prior)

In [19]:
likelihood_object.w_i(hyperparams_dic={'lambda1':1, 'lambda2':1, 'R':100})

KeyboardInterrupt: 

In [18]:
likelihood_object.lnlike(hyperparams_dic={'lambda1':1, 'lambda2':1, 'R':100})

KeyboardInterrupt: 

## Testing functions to create classes 

In [52]:
def create_uniform_hyperparam_prior_class(hyperparam_range_dic):
    class UniformHyperparamPrior(UniformPriorMixin, IdentityTransformMixin, Prior):
        def __init__(self):
            self.standard_params = list(hyperparam_range_dic)
            self.range_dic = hyperparam_range_dic
        def get_init_dict():
            return hyperparam_range_dic
    return UniformHyperparamPrior

In [53]:
UniformHyperparamPrior1 = create_uniform_hyperparam_prior_class(pop_prior.hyperparam_range_dic)

In [54]:
uniform_prior = UniformHyperparamPrior1()
uniform_prior.standard_params

['lambda1', 'lambda2']

In [68]:
def create_combined_hyperparam_class(hyperparam_prior_class_list):
    class CombinedHyperparamPrior(CombinedPrior):
        prior_classes = hyperparam_prior_class_list
    return CombinedHyperparamPrior

In [69]:
combined_prior_class = create_combined_hyperparam_class([FixedDistancePrior, FixedInclinationPrior, GaussianTestPrior])

In [65]:
combined_prior_class()

CombinedHyperparamPrior(m1, m2) → [d_luminosity, iota, m1, m2]

In [61]:
%debug

> /home/isha/PopulationSelectionEffects/cogwheel_population_inference/cogwheel/cogwheel/prior.py(600)_set_params()
    598         """
    599         cls.range_dic = {}
--> 600         for prior_class in cls.prior_classes:
    601             cls.range_dic.update(prior_class.range_dic)
    602 



ipdb>  cls.prior_classes


ipdb>  prior_class


*** NameError: name 'prior_class' is not defined
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


## Testing vecrtorized functions

In [ ]:
par_dict2 = {'m1':np.array([5,4,3,2,1]), 'm2':np.array([5,4,3,2,1]), 'lambda1':5, 'lambda2':5}
lnprior_func1 = np.vectorize(pop_prior.lnprior)
lnprior_func2 = np.vectorize(pop_prior2.lnprior)
lnprior_func1(**par_dict)

In [12]:
lnprior_func2(**par_dict2)

array([ 4.15358748e+00, -3.95846413e+02, -1.59584641e+03,
       -3.59584641e+03, -6.39584641e+03])

In [11]:
pop_prior2.lnprior(m1=5, m2=5, lambda1=5, lambda2=5)

4.153587480698636

In [17]:
inj_mass_prior.sampled_params

['m1_source', 'cum_q', 'd_hat']

In [22]:
issubclass(GaussianTestPopulationPrior, prior.Prior)

True

## Test creating ParametrizedPrior class

In [13]:
obj = GaussianHyperparams()

In [14]:
obj.standard_params

['lambda1', 'lambda2']

In [31]:
ex = []
for i in [['abc','b'], 'abc', 'b']:
    print(i)
    ex.append(i)
    print(ex)
ex

['abc', 'b']
[['abc', 'b']]
abc
[['abc', 'b'], 'abc']
b
[['abc', 'b'], 'abc', 'b']


[['abc', 'b'], 'abc', 'b']

In [32]:
set(ex)

TypeError: unhashable type: 'list'

## Test new CombinedParametrizedPrior class

In [7]:
combined_pop_prior = FixedTestPopulationPrior(f_ref=20) # combined prior
#pop_prior = GaussianTestParametrizedPrior() # population prior

In [8]:
inj_prior=InjectionPrior(detector_pair='HL', tgps=0.0, ref_det_name='H', f_avg=10, 
                         m1_source_range=(3,10),f_ref=20)

In [9]:
# now compute likelihood
injections_summary_dict = {'Ninj':20000, 'recovered_injections':injs_summary, 'pastro_func':np.ones(10)}
R0 = 70

likelihood_object = PopulationLikelihood(combined_pop_prior, injections_summary_dict, 
                                         all_events_pe, R0, injection_population_model=inj_prior)

In [12]:
likelihood_object.lnlike(hyperparams_dic={'lambda1':1, 'lambda2':1, 'R':100})

TypeError: lnprior() missing 2 required positional arguments: 'lambda1' and 'lambda2'

In [ ]:
%debug

> /home/isha/PopulationSelectionEffects/cogwheel_population_inference/cogwheel/cogwheel/population_inference/population_models.py(217)lnprior_and_transform()
    215                                          + subprior.conditioned_on
    216                                          + hyperparam_list)}
--> 217                 lnp += subprior.lnprior(**input_dic)
    218             return lnp, standard_par_dic
    219 



ipdb>  input_dic


{'m1': 5.431003254248099, 'm2': 5.417386225076431}
